## Dunbrack cluster assignment

This notebook assigns structures (e.g. a pdb file or a simulation trajectory) into Dunbrack clusters based on [Modi and Dunbrack, 2019](https://pubmed.ncbi.nlm.nih.gov/30867294/).

Maintainer: [@jiayeguo](https://github.com/jiayeguo)

In [1]:
import mdtraj as md
import numpy as np
from klifs import Klifs
from query_klifs import query_klifs_database
import protein
from math import cos

#### set pdbid and chain id for the structure of interest

In [2]:
pdbid = '2JIU'
chain = 'B'

#### load the structure using [MDTraj](http://mdtraj.org/1.9.3/).

Use the function [load_pdb](http://mdtraj.org/1.9.3/api/generated/mdtraj.load_pdb.html?highlight=load_pdb#mdtraj.load_pdb) for pdb structures and [load_dcd](http://mdtraj.org/1.9.3/api/generated/mdtraj.load_dcd.html?highlight=load_dcd#mdtraj.load_dcd) for simulations trajectories.

In [3]:
traj = md.load_pdb(f'./{pdbid}_chain{chain}.pdb')

#### get key residue indices to calculate structural features for cluster assignment 

In [4]:
klifs = query_klifs_database(pdbid, chain)
key_res = protein.key_klifs_residues(klifs.numbering)

#### assign the Dunbrack cluster based on dihedrals and distances ([Modi and Dunbrack, 2019](https://pubmed.ncbi.nlm.nih.gov/30867294/))

Notice that the distance unit output from MDTraj might be different given different input filetypes. The distances are in:

- Angstroms (when dcd files from regular MD simulations are input into MDTraj)
- nanometers (when pdb files or dcd files from SAMS simulations are input into MDTraj)

#### define the centroid of each dihedral for clusters 0-5

In [5]:
centroid = dict()
centroid[(0, 'x_phi')]=  -129.0
centroid[(0, 'x_psi')]=   179.0
centroid[(0, 'd_phi')]=    61.0
centroid[(0, 'd_psi')]=    81.0
centroid[(0, 'f_phi')]=   -97.0
centroid[(0, 'f_psi')]=    20.0
centroid[(0, 'f_chi1')]=  -71.0

centroid[(1, 'x_phi')]=  -119.0
centroid[(1, 'x_psi')]=   168.0
centroid[(1, 'd_phi')]=    59.0
centroid[(1, 'd_psi')]=    34.0
centroid[(1, 'f_phi')]=   -89.0
centroid[(1, 'f_psi')]=    -8.0
centroid[(1, 'f_chi1')]=   56.0

centroid[(2, 'x_phi')]=  -112.0
centroid[(2, 'x_psi')]=    -8.0
centroid[(2, 'd_phi')]=  -141.0
centroid[(2, 'd_psi')]=   148.0
centroid[(2, 'f_phi')]=  -128.0
centroid[(2, 'f_psi')]=    23.0
centroid[(2, 'f_chi1')]=  -64.0

centroid[(3, 'x_phi')]=  -135.0
centroid[(3, 'x_psi')]=   175.0
centroid[(3, 'd_phi')]=    60.0
centroid[(3, 'd_psi')]=    65.0
centroid[(3, 'f_phi')]=   -79.0
centroid[(3, 'f_psi')]=   145.0
centroid[(3, 'f_chi1')]=  -73.0

centroid[(4, 'x_phi')]=  -125.0
centroid[(4, 'x_psi')]=   172.0
centroid[(4, 'd_phi')]=    60.0
centroid[(4, 'd_psi')]=    33.0
centroid[(4, 'f_phi')]=   -85.0
centroid[(4, 'f_psi')]=   145.0
centroid[(4, 'f_chi1')]=   49.0

centroid[(5, 'x_phi')]=  -106.0
centroid[(5, 'x_psi')]=   157.0
centroid[(5, 'd_phi')]=    69.0
centroid[(5, 'd_psi')]=    21.0
centroid[(5, 'f_phi')]=   -62.0
centroid[(5, 'f_psi')]=   134.0
centroid[(5, 'f_chi1')]= -145.0

#### calculate the dihedrals and distances for Dunbrack cluster assignment

In [6]:
dihedrals, distances = protein.compute_simple_protein_features(traj, key_res)

There is no missing coordinates.  All dihedrals and distances will be computed.


In [7]:
assignment = list()
for i in range(len(distances)):
    ## reproduce the Dunbrack clustering
    ## level1: define the DFG positions
    if distances[i][0] <= (11.0 / 10) and distances[i][1] <= (11.0 / 10):
        ## can only be BABtrans
        assignment.append(7)
    elif distances[i][0] > (11.0 / 10) and distances[i][1] < (14.0 / 10):
        ## can only be BBAminus
        assignment.append(6)
    else:
        ## belong to DFGin and possibly clusters 0 - 5
        mindist=10000.0
        cluster_assign = 0

        for c in range(6):
            total_dist = float((2.0 * (1.0-cos((dihedrals[i][0] - centroid[(c, 'x_phi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][1] - centroid[(c, 'x_psi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][2] - centroid[(c, 'd_phi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][3] - centroid[(c, 'd_psi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][4] - centroid[(c, 'f_phi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][5] - centroid[(c, 'f_psi')])*np.pi / 180.0)))
            + (2.0 * (1.0-cos((dihedrals[i][6] - centroid[(c, 'f_chi1')])*np.pi / 180.0)))) / 7
            if total_dist < mindist:
                mindist = total_dist
                clust_assign = c
        assignment.append(clust_assign)

In [8]:
print(f"The input structure is assigned to Dunbrack cluster(s): {assignment}")

The input structure is assigned to Dunbrack cluster(s): [7]
